# Pivot Tables

Note these links lead to the Github wiki pages for this section


[**Preface**](https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#preface)
* [Anatomy of the `pivot_table()` command](https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#anatomy-of-the-pivot_table-command)

[**Calculating count, distinct count, sum, and mean with `aggfunc`**](https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#calculating-count-distinct-count-sum-and-mean-with-aggfunc)
* [Single Aggregation]( https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#single-aggregation)
* [Multiple Aggregations]( https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#multiple-aggregations)
* [Adding multiple indices, columns, and values](https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#adding-multiple-indices-columns-and-values)
* [Using `fill_value` to replace `NaN`](https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#using-fill_value-to-replace-nan)
* [Adding totals to your table using `margins`](https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#adding-totals-to-your-table-using-margins)
* [Multiple `aggfunc` variables](https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#multiple-aggfunc-variables)

[**Chaining Commands**](https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#chaining-commands)
* [Changing the index into columns - `reset_index()`](https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#changing-the-index-into-columns---reset_index)
* [Renaming column names - `rename()`](https://github.com/kn-kn/python-guide/wiki/Pivot-Tables#renaming-column-names---rename)


## Preface

The `pivot_table()` function on Pandas is very similar to other pivot tables found on other software like Microsoft Excel. [Practical Business Python](https://pbpython.com) provides a very simple guide to using the command. This section is inspired from their site, so credits to them!

This page uses [this file](https://pbpython.com/extras/sales-funnel.xlsx) found on the Practical Business Python site. A preview is shown below.

`df.head(5)`

| Account | Name                         | Rep           | Manager      | Product     | Quantity | Price | Status    |
|---------|------------------------------|---------------|--------------|-------------|----------|-------|-----------|
| 714466  | Trantow-Barrows              | Craig Booker  | Debra Henley | CPU         | 1        | 30000 | presented |
| 714466  | Trantow-Barrows              | Craig Booker  | Debra Henley | Software    | 1        | 10000 | presented |
| 714466  | Trantow-Barrows              | Craig Booker  | Debra Henley | Maintenance | 2        | 5000  | pending   |
| 737550  | Fritsch, Russel and Anderson | Craig Booker  | Debra Henley | CPU         | 1        | 35000 | declined  |
| 146832  | Kiehn-Spinka                 | Daniel Hilton | Debra Henley | CPU         | 2        | 65000 | won       |

This page loosely follows this [guide](https://pbpython.com/pandas-pivot-table-explained.html).

### Anatomy of the `pivot_table()` command

[Documentation for `Pivot_Table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

`pivot_table()` has a lot of optional parameters that can be added but we will be focusing only on a few. The list of parameters below are the more common ones you'll most likely use.

`pd.pivot_table(data, index=[], columns=[], values=[], aggfunc={}, fill_values=0, margins=False, margins_name=False, dropna=False)`

* **data**: the data for the pivot table, most likely a dataframe
* **index**: the index of your dataframe; i.e. the rows
* **columns**: the columns of your dataframe (note this is optional)
* **values**: the columns you want to run aggregate commands like sum or mean
* **aggfunc**: where you specify your aggregate commands
* **fill_values**: what to fill missing values with
* **margins**: if you want totals
* **margins_name**: the name of your totals row/column, if it exists
* **dropna**: ignore columns that are completely NaN

![Anatomy of Pivot Table](https://pbpython.com/images/pivot-table-datasheet.png)

## Calculating count, distinct count, sum, and mean with `aggfunc`

The following are the `aggfunc` commands for each aggregation:

* **Count**: `len`
* **Distinct Count**: `lambda x: len(x.unique())`
* **Sum**: `np.sum`
* **Mean**: `np.mean`

### Single Aggregation

Here is an example of using `np.sum` to find the total price for each firm:

In [1]:
import pandas as pd
import numpy as np

file_path = "C:/Users/kenguyen/Downloads/sales-funnel.xlsx"
df = pd.read_excel(file_path)

In [2]:
df.head(5)

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [3]:
pd.pivot_table(df, index=['Name'], values=['Price'], aggfunc=np.sum)

,Price
Name,
Barton LLC,35000
"Fritsch, Russel and Anderson",35000
Herman LLC,65000
Jerde-Hilpert,5000
"Kassulke, Ondricka and Metz",7000
Keeling LLC,100000
Kiehn-Spinka,65000
Koepp Ltd,70000
Kulas Inc,50000


Here is an example of using `lambda x: len(x.unique())` to find the number of unique firms who bought each product:

In [4]:
pd.pivot_table(df, index=['Product'], values=['Name'], aggfunc=lambda x: len(x.unique()))

,Name
Product,
CPU,9
Maintenance,4
Monitor,1
Software,3


### Multiple Aggregations

You can add multiple aggregations to calculate more than one metric at a time. Below we have sum, mean, and count:

In [5]:
pd.pivot_table(df, index=['Name'], values=['Price'], aggfunc=[np.sum, np.mean, len])

,sum,mean,len
,Price,Price,Price
Name,,,
Barton LLC,35000,35000,1
"Fritsch, Russel and Anderson",35000,35000,1
Herman LLC,65000,65000,1
Jerde-Hilpert,5000,5000,1
"Kassulke, Ondricka and Metz",7000,7000,1
Keeling LLC,100000,100000,1
Kiehn-Spinka,65000,65000,1
Koepp Ltd,70000,35000,2


### Adding multiple indices, columns, and values

You can add columns using the parameter `columns=` to further split your data:

In [6]:
pd.pivot_table(df, index=['Name'], values=['Price', 'Quantity'], columns=['Product'], aggfunc=[np.sum])

sum                                        \
                                 Price                              Quantity   
Product                            CPU Maintenance Monitor Software      CPU   
Name                                                                           
Barton LLC                     35000.0         NaN     NaN      NaN      1.0   
Fritsch, Russel and Anderson   35000.0         NaN     NaN      NaN      1.0   
Herman LLC                     65000.0         NaN     NaN      NaN      2.0   
Jerde-Hilpert                      NaN      5000.0     NaN      NaN      NaN   
Kassulke, Ondricka and Metz        NaN      7000.0     NaN      NaN      NaN   
Keeling LLC                   100000.0         NaN     NaN      NaN      5.0   
Kiehn-Spinka                   65000.0         NaN     NaN      NaN      2.0   
Koepp Ltd                      65000.0         NaN  5000.0      NaN      2.0   
Kulas Inc                      40000.0         NaN     NaN  10000.0      2.0   
Purdy-Kunde                    30000.0         NaN     NaN      NaN      1.0   
Stokes LLC                         NaN      5000.0     NaN  10000.0      NaN   
Trantow-Barrows                30000.0      5000.0     NaN  10000.0      1.0   

                                                           
                                                           
Product                      Maintenance Monitor Software  
Name                                                       
Barton LLC                           NaN     NaN      NaN  
Fritsch, Russel and Anderson         NaN     NaN      NaN  
Herman LLC                           NaN     NaN      NaN  
Jerde-Hilpert                        2.0     NaN      NaN  
Kassulke, Ondricka and Metz          3.0     NaN      NaN  
Keeling LLC                          NaN     NaN      NaN  
Kiehn-Spinka                         NaN     NaN      NaN  
Koepp Ltd                            NaN     2.0      NaN  
Kulas Inc                            NaN     NaN      1.0  
Purdy-Kunde                          NaN     NaN      NaN  
Stokes LLC                           1.0     NaN      1.0  
Trantow-Barrows                      2.0     NaN      1.0

### Using `fill_value` to replace `NaN`

The NaN makes it very hard to read. We can add the `fill_value` parameter to specify what NaN should be replaced with. Lets replace them with blanks:

In [7]:
pd.pivot_table(df, index=['Name'], values=['Price', 'Quantity'], columns=['Product'], aggfunc=[np.sum], fill_value="")

sum                                        \
                               Price                              Quantity   
Product                          CPU Maintenance Monitor Software      CPU   
Name                                                                         
Barton LLC                     35000                                     1   
Fritsch, Russel and Anderson   35000                                     1   
Herman LLC                     65000                                     2   
Jerde-Hilpert                               5000                             
Kassulke, Ondricka and Metz                 7000                             
Keeling LLC                   100000                                     5   
Kiehn-Spinka                   65000                                     2   
Koepp Ltd                      65000                5000                 2   
Kulas Inc                      40000                        10000        2   
Purdy-Kunde                    30000                                     1   
Stokes LLC                                  5000            10000            
Trantow-Barrows                30000        5000            10000        1   

                                                           
                                                           
Product                      Maintenance Monitor Software  
Name                                                       
Barton LLC                                                 
Fritsch, Russel and Anderson                               
Herman LLC                                                 
Jerde-Hilpert                          2                   
Kassulke, Ondricka and Metz            3                   
Keeling LLC                                                
Kiehn-Spinka                                               
Koepp Ltd                                      2           
Kulas Inc                                               1  
Purdy-Kunde                                                
Stokes LLC                             1                1  
Trantow-Barrows                        2                1

Sometimes you have to move your columns and indices around in order for your table to make sense. If we move our column into the index, it considerably improves comprehensibility of our table:

In [8]:
pd.pivot_table(df, index=['Name', 'Product'], values=['Price', 'Quantity'], aggfunc=[np.sum], fill_value="")

sum         
                                           Price Quantity
Name                         Product                     
Barton LLC                   CPU           35000        1
Fritsch, Russel and Anderson CPU           35000        1
Herman LLC                   CPU           65000        2
Jerde-Hilpert                Maintenance    5000        2
Kassulke, Ondricka and Metz  Maintenance    7000        3
Keeling LLC                  CPU          100000        5
Kiehn-Spinka                 CPU           65000        2
Koepp Ltd                    CPU           65000        2
                             Monitor        5000        2
Kulas Inc                    CPU           40000        2
                             Software      10000        1
Purdy-Kunde                  CPU           30000        1
Stokes LLC                   Maintenance    5000        1
                             Software      10000        1
Trantow-Barrows              CPU           30000        1
                             Maintenance    5000        2
                             Software      10000        1

### Adding totals to your table using `margins`

You can easily sum up the rows or columns of your pivot table by adding the parameter `margins=True`:

In [9]:
pd.pivot_table(df, index=['Name', 'Product'], values=['Price', 'Quantity'], aggfunc=[np.sum], fill_value="", margins=True)

sum         
                                           Price Quantity
Name                         Product                     
Barton LLC                   CPU           35000        1
Fritsch, Russel and Anderson CPU           35000        1
Herman LLC                   CPU           65000        2
Jerde-Hilpert                Maintenance    5000        2
Kassulke, Ondricka and Metz  Maintenance    7000        3
Keeling LLC                  CPU          100000        5
Kiehn-Spinka                 CPU           65000        2
Koepp Ltd                    CPU           65000        2
                             Monitor        5000        2
Kulas Inc                    CPU           40000        2
                             Software      10000        1
Purdy-Kunde                  CPU           30000        1
Stokes LLC                   Maintenance    5000        1
                             Software      10000        1
Trantow-Barrows              CPU           30000        1
                             Maintenance    5000        2
                             Software      10000        1
All                                       522000       30

### Multiple `aggfunc` variables

If you leave `aggfunc` as a list, it'll apply every single aggregation to all your values. If you want to specific which aggregation belongs to which value, use a dictionary {} instead.

Below, we specify to calculate the mean only on price and sum only on quantity.

In [10]:
pd.pivot_table(df, index=['Name', 'Product'], values=['Price', 'Quantity'], aggfunc={'Price': np.mean, 'Quantity': np.sum})

Price  Quantity
Name                         Product                      
Barton LLC                   CPU           35000         1
Fritsch, Russel and Anderson CPU           35000         1
Herman LLC                   CPU           65000         2
Jerde-Hilpert                Maintenance    5000         2
Kassulke, Ondricka and Metz  Maintenance    7000         3
Keeling LLC                  CPU          100000         5
Kiehn-Spinka                 CPU           65000         2
Koepp Ltd                    CPU           65000         2
                             Monitor        5000         2
Kulas Inc                    CPU           40000         2
                             Software      10000         1
Purdy-Kunde                  CPU           30000         1
Stokes LLC                   Maintenance    5000         1
                             Software      10000         1
Trantow-Barrows              CPU           30000         1
                             Maintenance    5000         2
                             Software      10000         1

## Chaining Commands

You can also chain other panda commands onto your pivot table. This can easily allow you to customize your pivot table all in a single statement.

---

### Changing the index into columns - `reset_index()`

By default, index values remain in the index. You can call `reset_index()` to set them as columns instead. This may result in more legibility depending on your situation.

In [11]:
pd.pivot_table(df, index=['Name', 'Product'], values=['Price', 'Quantity'], 
               aggfunc={'Price': np.mean, 'Quantity': np.sum}).reset_index()

,Name,Product,Price,Quantity
0,Barton LLC,CPU,35000,1
1,"Fritsch, Russel and Anderson",CPU,35000,1
2,Herman LLC,CPU,65000,2
3,Jerde-Hilpert,Maintenance,5000,2
4,"Kassulke, Ondricka and Metz",Maintenance,7000,3
5,Keeling LLC,CPU,100000,5
6,Kiehn-Spinka,CPU,65000,2
7,Koepp Ltd,CPU,65000,2
8,Koepp Ltd,Monitor,5000,2
9,Kulas Inc,CPU,40000,2


### Renaming column names - `rename()`

Rename columns by adding `rename()` to your statement.

In [12]:
pd.pivot_table(df, index=['Name', 'Product']
               , values=['Price', 'Quantity']
               , aggfunc={'Price': np.mean, 'Quantity': np.sum}).reset_index().rename(
                 columns={'Name': 'Lawyer Firm', 'Product': 'Type of Service'})

,Lawyer Firm,Type of Service,Price,Quantity
0,Barton LLC,CPU,35000,1
1,"Fritsch, Russel and Anderson",CPU,35000,1
2,Herman LLC,CPU,65000,2
3,Jerde-Hilpert,Maintenance,5000,2
4,"Kassulke, Ondricka and Metz",Maintenance,7000,3
5,Keeling LLC,CPU,100000,5
6,Kiehn-Spinka,CPU,65000,2
7,Koepp Ltd,CPU,65000,2
8,Koepp Ltd,Monitor,5000,2
9,Kulas Inc,CPU,40000,2
